# COVID19 Analysis

In [50]:
from sqlalchemy.engine import create_engine

import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import hvplot.pandas  
%matplotlib inline
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression
#from sklearn import metrics
import sys
import datetime

## BigQuery with SQLAlchemy to Pandas Dataframe

In [51]:
# Establish the engine

bigquery_uri = 'bigquery://storied-port-224520/covid'

engine = create_engine(
    bigquery_uri,
    credentials_path='/Users/heath/ssh/storied-port-224520-4013c7b6d6d9.json'
)

### Queries

In [52]:
def dfall():
    query = """
            SELECT
                *
            FROM
              covid.wd
            """

    rows = engine.execute(query).fetchall()
    rows = [dict(row) for row in rows]

    dfqall1 = pd.DataFrame(list(rows))
    return dfqall1

def DfNewTests():
    query = """
            SELECT
              wd.date AS date,
              wd.location AS location,
              SUM(wd.new_tests) AS sum_new_tests
            FROM
              covid.wd
            GROUP BY
              location,
              date
            """

    rows = engine.execute(query).fetchall()
    rows = [dict(row) for row in rows]

    df = pd.DataFrame(list(rows))
    return df

def DfNewCases():
    query = """
            SELECT
              wd.date AS date,
              wd.location AS location,
              SUM(wd.new_cases) AS sum_new_cases
            FROM
              covid.wd
            GROUP BY
              location,
              date
            """
#             WHERE
#              ((DATETIME_TRUNC(CAST(wd.date AS DATETIME), DAY) > DATETIME_TRUNC(DATETIME('2021-05-01'), DAY)) AND (DATETIME_TRUNC(CAST(wd.date AS DATETIME), DAY) < DATETIME_TRUNC(DATETIME('2021-06-01'), DAY)))
            
    rows = engine.execute(query).fetchall()
    rows = [dict(row) for row in rows]

    df = pd.DataFrame(list(rows))
    return df

def DfnewTestsCases():
    query = """
                SELECT
                  wd.date AS date,
                  wd.location AS location,
                  SUM(wd.new_tests) AS sum_new_tests,
                  SUM(wd.new_cases) AS sum_new_cases,
                  SUM(wd.new_vaccinations) AS sum_new_vaccinations
                FROM
                  covid.wd
                WHERE
                  ((DATETIME_TRUNC(CAST(wd.date AS DATETIME), DAY) > DATETIME_TRUNC(DATETIME('2021-01-01'), DAY)) AND (DATETIME_TRUNC(CAST(wd.date AS DATETIME), DAY) < DATETIME_TRUNC(DATETIME('2021-05-31'), DAY)))
                GROUP BY
                  date,
                  location
            """           

    rows = engine.execute(query).fetchall()
    rows = [dict(row) for row in rows]

    df = pd.DataFrame(list(rows))
    return df

### Create DataFrames

In [53]:
dfnewtestscases = DfnewTestsCases()
dfnewtestscases.head(1)


/Users/heath/opt/anaconda3/lib/python3.8/site-packages/google/cloud/bigquery/client.py:512: UserWarning:

Dependency google-cloud-bigquery-storage is outdated, please upgrade it to version >= 2.0.0 (version found: legacy).



,date,location,sum_new_tests,sum_new_cases,sum_new_vaccinations
0,2021-01-02,Africa,NaN,22919.0,NaN


In [30]:
#dfnewtests = DfNewTests()
#dfnewtests.head(1)

In [31]:
#dfnewcases = DfNewCases()
#dfnewcases.head(1)

In [54]:
usa = dfnewtestscases[dfnewtestscases['location'].str.contains('United States')].reset_index()
usa_tc = usa.copy()

## Data Statistics

In [55]:
dfnewtestscases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32184 entries, 0 to 32183
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  32184 non-null  object 
 1   location              32184 non-null  object 
 2   sum_new_tests         13986 non-null  float64
 3   sum_new_cases         29793 non-null  float64
 4   sum_new_vaccinations  10610 non-null  float64
dtypes: float64(3), object(2)
memory usage: 1.2+ MB


In [56]:
dfnewtestscases['location'].value_counts().head(20)

Costa Rica          149
Namibia             149
Argentina           149
Papua New Guinea    149
Iraq                149
World               149
Malaysia            149
Taiwan              149
Portugal            149
Afghanistan         149
Italy               149
Comoros             149
Bangladesh          149
South Sudan         149
Finland             149
United Kingdom      149
Latvia              149
United States       149
Eswatini            149
South Africa        149
Name: location, dtype: int64

In [57]:
usa = dfnewtestscases[dfnewtestscases['location'].str.contains('United States')].reset_index()
#usa_tests_org = dfnewtestscases[dfnewtestscases['location'].str.contains('United States')].reset_index()


In [58]:
mx = dfnewtestscases[dfnewtestscases['location'].str.contains('Mexico')].reset_index()

In [37]:
#mx['DateTime'] = (mx.Date) + ' ' + (mx.Time)


In [60]:
mx.head()

,index,date,location,sum_new_tests,sum_new_cases,sum_new_vaccinations
0,29733,2021-01-02,Mexico,14875.0,6359.0,NaN
1,29734,2021-01-03,Mexico,9385.0,5211.0,NaN
2,29735,2021-01-04,Mexico,42832.0,6464.0,NaN
3,29736,2021-01-05,Mexico,43981.0,11271.0,4276.0
4,29737,2021-01-06,Mexico,43374.0,13345.0,4949.0


In [61]:
mx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 149 non-null    int64  
 1   date                  149 non-null    object 
 2   location              149 non-null    object 
 3   sum_new_tests         149 non-null    float64
 4   sum_new_cases         149 non-null    float64
 5   sum_new_vaccinations  129 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 7.1+ KB


## Data Prep

In [62]:
# datetime conversion
usa[["date"]] = usa[["date"]].astype('datetime64')

In [63]:
# datetime conversion
mx[["date"]] = mx[["date"]].astype('datetime64')

In [64]:
usa['month'] = usa['date'].apply(lambda time: time.month)
usa['day'] = usa['date'].apply(lambda time: time.dayofweek)
usa['hour'] = usa['date'].apply(lambda time: time.hour)

dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
mmap = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
usa['day'] = usa['day'].map(dmap)
usa['month'] = usa['month'].map(mmap)

In [65]:
mx['month'] = mx['date'].apply(lambda time: time.month)
mx['day'] = mx['date'].apply(lambda time: time.dayofweek)
mx['hour'] = mx['date'].apply(lambda time: time.hour)

dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
mmap = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
mx['day'] = mx['day'].map(dmap)
mx['month'] = mx['month'].map(mmap)

In [66]:
mx.head(1)

,index,date,location,sum_new_tests,sum_new_cases,sum_new_vaccinations,month,day,hour
0,29733,2021-01-02,Mexico,14875.0,6359.0,NaN,Jan,Sat,0


In [45]:
#usa.info()

In [67]:
usa_tc.drop(columns=['date', 'index', 'location'], inplace=True)
usa_tc.head()

,sum_new_tests,sum_new_cases,sum_new_vaccinations
0,1108645.0,300462.0,NaN
1,1041804.0,208853.0,NaN
2,1393390.0,184005.0,NaN
3,1953397.0,235042.0,273209.0
4,2319417.0,255637.0,470328.0


In [47]:

#mx_tc.head()

In [48]:
#mx_tc.isna().sum()

# Data Analysis

### 4 Day Moving Average

In [68]:
mx_tc = mx.copy()
mx_tc.drop(columns=['location', 'sum_new_tests', 'sum_new_vaccinations', 'index'], inplace=True)
mx_tc.index = mx_tc.date
mx_tc['CMA_4'] = mx_tc.expanding(min_periods=4).mean()

plt.figure(figsize=[15,10])
plt.grid(True)
plt.plot(mx_tc['sum_new_cases'],label='new Cases')
plt.plot(mx_tc['CMA_4'],label='CMA_4')
plt.legend(loc=2)

ValueError: Wrong number of items passed 2, placement implies 1

### New Tests, Cases, Vaccinations

In [69]:

x = usa.hvplot.line(
    x='date', 
    y='sum_new_tests', 
    xlabel='Date', 
    ylabel='Tests', 
    label='New Tests - United States', 
    width=900, 
    height=400
)
y = usa.hvplot.line(
    x='date', 
    y='sum_new_cases', 
    xlabel='Date', 
    ylabel='Cases', 
    label='New Cases - United States', 
    width=900, 
    height=400
)
z = usa.hvplot.line(
    x='date', 
    y='sum_new_vaccinations', 
    xlabel='Date', 
    ylabel='Vaccinations', 
    label='Vaccinations - United States', 
    width=900, 
    height=400
)

x+y+z

:Layout
   .Curve.New_Tests_hyphen_minus_United_States    :Curve   [date]   (sum_new_tests)
   .Curve.New_Cases_hyphen_minus_United_States    :Curve   [date]   (sum_new_cases)
   .Curve.Vaccinations_hyphen_minus_United_States :Curve   [date]   (sum_new_vaccinations)

### All - Tests, Cases, Vaccinations

In [ ]:
usa.hvplot(
    x='date', 
    y=['sum_new_tests', 'sum_new_cases', 'sum_new_vaccinations'], 
    value_label='rate',
    height=500, 
    width=900,
    legend='top_right'
)


### Tests - Day by Month

In [ ]:
usa.hvplot.heatmap(
    x='month', 
    y='date.day', 
    C='sum_new_tests',
    title='Tests - Day by Month',
    height=500, 
    width=600, 
    colorbar=True)

### Tests - Day by Month Grouped by Day

In [ ]:
usa.hvplot.heatmap(
    x='month', 
    y='day', 
    C='sum_new_tests',
    title='Tests - Day by Month Grouped by Day',
    height=500, 
    width=600, 
    colorbar=True)

### Vaccinations - Day/Month Grouped by Day

In [ ]:
usa.hvplot.heatmap(
    x='month', 
    y='day', 
    C='sum_new_vaccinations',
    title='Vaccinations - Day/Month Grouped by Day',
    height=500, 
    width=600, 
    colorbar=True)

### Pair Plot

In [ ]:
pairplot = sns.pairplot(usa_tc.dropna())

### Tests Histogram

In [ ]:
usa.hvplot.hist("sum_new_tests", by="location", subplots=True, width=600)

### Box Plots by Month

In [ ]:
a = usa.hvplot.box(y='sum_new_tests', by='month', height=500, width=500, title='Tests', legend=False)
b = usa.hvplot.box(y='sum_new_cases', by='month', height=500, width=500, title='Cases', legend=False)
c = usa.hvplot.box(y='sum_new_vaccinations', by='month', height=500, width=500, title='Vaccinations', legend=False)
a+b+c

### Regression Plot - tests vs cases

In [ ]:
ax = sns.relplot(
    x="sum_new_tests", 
    y="sum_new_cases",
    hue="sum_new_vaccinations", 
    data=usa, 
    height=8,
    legend=True
)

ax.set(
    xlabel='New Tests',
    ylabel='New Cases',
    title='Tests vs Cases'
)


In [ ]:
a = mx_tc.hvplot.line(
    x='date', 
    #y='sum_new_cases',
    y="CMA_4",
    xlabel='Date', 
    ylabel='Cases', 
    label='New Tests - Mexico', 
    width=900, 
    height=400
)

b = mx_tc.hvplot.line(
    x='date', 
    y='sum_new_cases',
    xlabel='Date', 
    ylabel='Cases', 
    label='New Tests - Mexico', 
    width=900, 
    height=400
)

a*b